In [1]:
import json
from IPython.display import HTML


# FIXME> _repr_
def inline_map(m, width=750, height=500):
    """Takes a Folium map instance and return an iframe."""
    m._build_map()
    srcdoc = m.HTML

    # FIXME: Needs the same for plug-in data.
    if m.json_data:
        callback = 'function(callback){{callback(null, JSON.parse({}))}}'
        for path, data in m.json_data.items():
            json_data = json.dumps(data).replace('"', '&quot;')
            srcdoc = srcdoc.replace('d3.json, {}'.format(repr(path)),
                                    callback.format((repr(json_data))))

    srcdoc = srcdoc.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{srcdoc}" '
                 'style="width: 100%; height: {height}px; '
                 'border: none"></iframe>'.format(srcdoc=srcdoc,
                                                  height=height))
    return embed


In [2]:
import folium

print(folium.__file__)
print(folium.__version__)

/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/folium/__init__.py
0.1.6


In [3]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

LAND = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                    edgecolor='face',
                                    facecolor=cfeature.COLORS['land'])

def make_map(bbox, projection=ccrs.Mercator()):
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw=dict(projection=projection))
    ax.set_extent(bbox)
    ax.add_feature(LAND, facecolor='0.25')
    ax.coastlines(resolution='50m')
    return fig, ax

In [4]:
import numpy.ma as ma
from scipy.io import loadmat
from oceans.colormaps import cm

merc = loadmat('./data/mercator_temperature.mat', squeeze_me=True)

x, y = merc['x'], merc['y']
bbox = [x.min(), x.max(), y.min(), y.max()]

level = 10  # 10 meters temperature.
data = ma.masked_invalid(merc['temp'][level, ...])

fig, ax = make_map(bbox=bbox)
cs = ax.pcolormesh(x, y, data, cmap=cm.avhrr, transform=ccrs.PlateCarree())
fig.savefig('GS.png', transparent=True)

We need to find a better way to trim the white space added by Cartopy.
Note that I left the land on purpose just check if the projection was OK.

In [5]:
!convert GS.png -trim gulf_stream.png

In [6]:
mapa = folium.Map(location=[y.mean(), x.mean()], zoom_start=5)

mapa.image_overlay('gulf_stream.png',
                   min_lat=y.min(), max_lat=y.max(),
                   min_lon=x.min(), max_lon=x.max(),
                   data_projection=False, image_opacity=0.5)

inline_map(mapa)